In [119]:
from pandas import read_excel, DataFrame, concat
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, f_oneway, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sb
import sys
import os
import numpy as np
import pingouin as pg
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import all_test,  equal_variance_test,normality_test, independence_test


# 연습문제 (3) - 분산분석
## 문제 1
유아들을 대상으로 세 가지 읽는 방법을 비교,실험하여 다음과 같은 독서평가 점수자료를 얻었다.

세 방법에 통계적으로 유의한 차이가 있는지 검정하라.

In [120]:
df = read_excel("https://data.hossam.kr/E02/reading.xlsx")
df

,독서방법,점수
0,B,6
1,B,9
2,B,2
3,B,16
4,D,7
5,D,7
6,D,12
7,D,10
8,D,16
9,S,11


먼저 데이터가 정규성,등분산성, 독립성을 따르는지 확인한다.

In [121]:
df2 = df.copy()
df2['독서방법'] = df2['독서방법'].map({'B': 0, 'D': 1, 'S': 2})
normality_test(df2['점수'])

C:\Users\yis82\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


statistic   p-value  result
condition test       field                               
정규성       shapiro    점수        0.925500  0.263591    True
          normaltest 점수        1.019042  0.600783    True
          ks_2samp   점수 & 점수   0.000000  1.000000    True

정규성을 위배하지 않는다.

In [122]:
equal_variance_test(df2[df2['독서방법']==0]['점수'],df2[df2['독서방법']==1]['점수'],df2[df2['독서방법']==2]['점수'])


statistic   p-value  result
condition test     field                                    
등분산성      Bartlett 점수 & 점수 & 점수   2.271441  0.321191    True
          Fligner  점수 & 점수 & 점수   2.115487  0.347239    True
          Levene   점수 & 점수 & 점수   1.448571  0.276420    True

위에 결과

등분산성을 위배하지 않는다.


|가설|내용|
|---|---|
|귀무가설|집단간 평균이 차이가 없다.(같다)|
|대립가설|집단간 평균의 차이가 있다.(다르다)|

In [123]:
reading_groups = df2.groupby('독서방법')

B_data = reading_groups.get_group(0)
D_data = reading_groups.get_group(1)
S_data = reading_groups.get_group(2)
B_data

,독서방법,점수
0,0,6
1,0,9
2,0,2
3,0,16


In [124]:
formula = '점수 ~ C(독서방법)'
lm = ols(formula, df2).fit()
anova = anova_lm(lm)
anova


,df,sum_sq,mean_sq,F,PR(>F)
C(독서방법),2.0,26.464286,13.232143,0.779403,0.48244
Residual,11.0,186.750000,16.977273,NaN,NaN


In [125]:
f_oneway(B_data['점수'],D_data['점수'],S_data['점수'])

F_onewayResult(statistic=0.7794033275960985, pvalue=0.4824404444119927)

p-value가 0.482이므로 우리가 가정한 유의수준 0.05보다 크므로 독서방법에 따른 점수의 평균은 차이가 없다는 귀무가설을 기각하기 위한 충분한 증거가 없기 떄문에 기각을 실패했다.

#### 문제 2

효소 성분의 비료가 효소 함량이 증가함에 따라 토마토 모종 포기의 성장에 어떤 영향을 주는지 실험연구를 하여 다음 자료를 얻었다.

분산분석을 하여 효소 함량에 따라 토마토 모종 성장에 차이가 나는지 검정하라.

In [126]:
df_enzyme = read_excel("https://data.hossam.kr/E02/tomato.xlsx")
df_enzyme

,효소함량,모종성장
0,0,10.8
1,0,9.1
2,0,13.5
3,0,9.2
4,1000,11.1
5,1000,11.2
6,1000,8.2
7,1000,11.3
8,5000,5.4
9,5000,4.6



|가설|내용|
|---|---|
|귀무가설|효소함량에 따른 모종성장의 평균이 차이가 없다.(같다)|
|대립가설|효소함량에 따른 모종성장의 평균의 차이가 있다.(다르다)|

In [127]:
normality_test(df_enzyme['모종성장'])

C:\Users\yis82\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


statistic   p-value  result
condition test       field                                   
정규성       shapiro    모종성장          0.954987  0.572547    True
          normaltest 모종성장          1.370910  0.503861    True
          ks_2samp   모종성장 & 모종성장   0.000000  1.000000    True

정규성을 위배하지 않는다.

In [128]:
equal_variance_test(df_enzyme[df_enzyme['효소함량']==0]['모종성장'],df_enzyme[df_enzyme['효소함량']==1000]['모종성장'],df_enzyme[df_enzyme['효소함량']==5000]['모종성장'],df_enzyme[df_enzyme['효소함량']==10000]['모종성장'])

statistic   p-value  result
condition test     field                                                 
등분산성      Bartlett 모종성장 & 모종성장 & 모종성장 & 모종성장   0.713261  0.870080    True
          Fligner  모종성장 & 모종성장 & 모종성장 & 모종성장   2.875138  0.411280    True
          Levene   모종성장 & 모종성장 & 모종성장 & 모종성장   0.310268  0.817616    True

등분산성을 위배하지 않는다.

One-way Anova를 이용해 가설검증을 실시한다.

In [129]:
pg.welch_anova(dv='모종성장', between='효소함량', data=df_enzyme)

,Source,ddof1,ddof2,F,p-unc,np2
0,효소함량,3,6.564072,10.932999,0.005925,0.751333


In [130]:
f_oneway(df_enzyme[df_enzyme['효소함량']==0]['모종성장'],df_enzyme[df_enzyme['효소함량']==1000]['모종성장'],df_enzyme[df_enzyme['효소함량']==5000]['모종성장'],df_enzyme[df_enzyme['효소함량']==10000]['모종성장'])

F_onewayResult(statistic=12.085748431431139, pvalue=0.0006149423417643626)

`p-value = 0.00006`가 설정한 유의수준인 0.05보다 작기 때문에 효소함량에 따라 평균 모종성장에 차이가 없다는 귀무가설을 기각할 충분한 증거가 있다고 말할 수 있다.

#### 문제 3
두 가지 교육방법을 비교하기 위하여 능력이 비슷한 고등학교 학생 22명을 추출하고, 11명씩 두 집단으로 나누어 한 학기 동안 각각의 교육방법으로 교육시킨 후 치른 학기말 시험 성적이 다음과 같다.

이 자료를 이용하여 두 가지 교육방법에 대한 국어와 영어의 효과가 서로 다르다고 할 수 있는지 검정하라. (분석 목적에 맞는 데이터 재배치 과정이 필요합니다.)

In [131]:
df_student = read_excel('https://data.hossam.kr/E02/edu.xlsx')

In [132]:
normality_test(df_student['국어'],df_student['영어'])



statistic   p-value  result
condition test       field                               
정규성       shapiro    국어        0.982805  0.954011    True
                     영어        0.953426  0.368662    True
          normaltest 국어        1.164455  0.558653    True
                     영어        1.589026  0.451801    True
          ks_2samp   국어 & 영어   0.136364  0.990057    True
                     영어 & 국어   0.136364  0.990057    True

In [133]:
equal_variance_test(df_student['국어'],df_student['영어'])

statistic   p-value  result
condition test     field                               
등분산성      Bartlett 국어 & 영어   0.204709  0.650946    True
          Fligner  국어 & 영어   0.517756  0.471800    True
          Levene   국어 & 영어   0.354049  0.555022    True

In [134]:
independence_test(df_student['국어'],df_student['영어'])

,,,statistic,p-value,result
condition,test,field,,,
독립성,Chi2,국어 & 영어,24.078968,0.289248,True


독립성을 위배하지않는다. 

따라서 이 데이터는 Two-way ANOVA 검증을 이용할 수 있다.

##### 데이터 전처리

In [135]:
df1 = df_student.filter(['교육방법','국어'])
df1.rename(columns= {'국어':'점수'}, inplace =True)
df1['과목'] = '국어'

df2 = df_student.filter(['교육방법','영어'])
df2.rename(columns= {'영어':'점수'}, inplace =True)
df2['과목'] = '영어'
df_edu = concat([df1,df2])
df_edu =df_edu.reset_index()
df_edu.drop(columns='index',inplace=True)
df_edu['과목'] = df_edu['과목'].map({'국어': 1, '영어': 2})
df_edu


,교육방법,점수,과목
0,1,65,1
1,1,87,1
2,1,73,1
3,1,79,1
4,1,81,1
5,1,69,1
6,1,55,1
7,1,76,1
8,1,77,1
9,1,70,1


|가설|내용|
|---|---|
|귀무가설|교육방법에 따른 과목간의 평균 성적에 차이가 없다.(같다)|
|대립가설|교육방법에 따른 과목간의 평균 성적에 차이가 있다.(다르다)|

In [136]:
formula = '점수 ~ C(교육방법)*C(과목)'
lm = ols(formula, df_edu).fit()
anova = anova_lm(lm)
anova

,df,sum_sq,mean_sq,F,PR(>F)
C(교육방법),1.0,355.113636,355.113636,3.845870,0.056857
C(과목),1.0,0.204545,0.204545,0.002215,0.962695
C(교육방법):C(과목),1.0,14.204545,14.204545,0.153835,0.696979
Residual,40.0,3693.454545,92.336364,NaN,NaN


교육방법에 따른 과목의 평균이 차이가 없음을 기각할 충분한 증거가 없어 기각하지 못했다.